<a href="https://colab.research.google.com/github/asteroiddestroyer911/Aryan_voice_AI/blob/main/Oceanopedia(chat_only).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 🌊 Marine Biodiversity AI Chatbot Training
# Complete setup for training with Llama 3.2 3B via Ollama

print("🌊 MARINE BIODIVERSITY AI CHATBOT TRAINING")
print("=" * 60)
print("🤖 Using Ollama Llama 3.2 3B Model")
print("📚 Training on AI in Ocean Conservation & Aquaculture Research")
print("=" * 60)

# Install all required packages
!pip install -q langchain langchain-community langchain-ollama
!pip install -q pypdf faiss-cpu sentence-transformers
!pip install -q streamlit pyngrok gradio
!pip install -q requests beautifulsoup4 numpy pandas

print("✅ All packages installed successfully!")


🌊 MARINE BIODIVERSITY AI CHATBOT TRAINING
🤖 Using Ollama Llama 3.2 3B Model
📚 Training on AI in Ocean Conservation & Aquaculture Research
✅ All packages installed successfully!


In [ ]:
# Install Ollama in Colab
import subprocess
import time
import requests
import json

print("🦙 Installing Ollama...")

# Download and install Ollama
!curl -fsSL https://ollama.ai/install.sh | sh

print("✅ Ollama installed!")

# Start Ollama server in background
import threading
import os

def start_ollama_server():
    """Start Ollama server in background"""
    subprocess.run(["ollama", "serve"], capture_output=False)

# Start server thread
server_thread = threading.Thread(target=start_ollama_server, daemon=True)
server_thread.start()

print("🔄 Starting Ollama server...")
time.sleep(10)

# Pull Llama 3.2 3B model
print("📥 Downloading Llama 3.2 3B model (this may take a few minutes)...")
!ollama pull llama3.2:3b

print("✅ Llama 3.2 3B model ready!")

# Test Ollama connection
try:
    response = requests.post('http://localhost:11434/api/generate',
                           json={'model': 'llama3.2:3b', 'prompt': 'Hello', 'stream': False})
    if response.status_code == 200:
        print("🎉 Ollama server running successfully!")
    else:
        print("⚠️ Ollama server connection issue")
except Exception as e:
    print(f"⚠️ Ollama setup error: {e}")


🦙 Installing Ollama...
>>> Cleaning up old version at /usr/local/lib/ollama
>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
#########                                                                 13.3%curl: (56) OpenSSL SSL_read: Connection reset by peer, errno 104


gzip: stdin: unexpected end of file
tar: Unexpected EOF in archive
tar: Unexpected EOF in archive
tar: Error is not recoverable: exiting now
✅ Ollama installed!
🔄 Starting Ollama server...
📥 Downloading Llama 3.2 3B model (this may take a few minutes)...

✅ Llama 3.2 3B model ready!
🎉 Ollama server running successfully!


In [ ]:
# Mount Google Drive and setup marine research data
from google.colab import drive
import os
import shutil
from pathlib import Path

print("🔗 Mounting Google Drive...")
drive.mount('/content/drive')
print("✅ Google Drive mounted successfully!")

# Setup directories
DRIVE_FOLDER = '/content/drive/MyDrive/Marine_Research_Data'
COLAB_DATA_DIR = '/content/marine_data'
VECTORSTORE_DIR = '/content/vectorstore'

# Create directories
os.makedirs(COLAB_DATA_DIR, exist_ok=True)
os.makedirs(VECTORSTORE_DIR, exist_ok=True)

print(f"📁 Created working directories:")
print(f"   • Marine Data: {COLAB_DATA_DIR}")
print(f"   • Vector Store: {VECTORSTORE_DIR}")

# Marine research PDF files
pdf_files = [
    'BIG-Data-SIH-1.pdf',  # AI in Ocean Conservation & Marine Research
    'BIG-Data-SIH-2.pdf',  # AI-driven aquaculture review
    'BIG-Data-SIH-3.pdf'   # GIS-based marine data integration
]

print(f"\n📋 Looking for PDFs in: {DRIVE_FOLDER}")
print("📄 Expected files:")
for pdf in pdf_files:
    print(f"   • {pdf}")

# Copy PDFs from Drive to Colab
copied_files = []
missing_files = []

for pdf_file in pdf_files:
    source_path = f"{DRIVE_FOLDER}/{pdf_file}"
    dest_path = f"{COLAB_DATA_DIR}/{pdf_file}"

    if os.path.exists(source_path):
        shutil.copy2(source_path, dest_path)
        copied_files.append(dest_path)
        print(f"✅ Copied: {pdf_file}")
    else:
        missing_files.append(pdf_file)
        print(f"❌ Missing: {pdf_file}")

print(f"\n📊 Summary:")
print(f"   • Copied: {len(copied_files)} files")
print(f"   • Missing: {len(missing_files)} files")

if missing_files:
    print(f"\n📝 To fix missing files:")
    print(f"   1. Upload the missing PDFs to: {DRIVE_FOLDER}")
    print(f"   2. Re-run this cell")
    print(f"   Missing files: {', '.join(missing_files)}")
else:
    print(f"\n🎉 All marine research PDFs ready for training!")


🔗 Mounting Google Drive...
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Google Drive mounted successfully!
📁 Created working directories:
   • Marine Data: /content/marine_data
   • Vector Store: /content/vectorstore

📋 Looking for PDFs in: /content/drive/MyDrive/Marine_Research_Data
📄 Expected files:
   • BIG-Data-SIH-1.pdf
   • BIG-Data-SIH-2.pdf
   • BIG-Data-SIH-3.pdf
❌ Missing: BIG-Data-SIH-1.pdf
❌ Missing: BIG-Data-SIH-2.pdf
❌ Missing: BIG-Data-SIH-3.pdf

📊 Summary:
   • Copied: 0 files
   • Missing: 3 files

📝 To fix missing files:
   1. Upload the missing PDFs to: /content/drive/MyDrive/Marine_Research_Data
   2. Re-run this cell
   Missing files: BIG-Data-SIH-1.pdf, BIG-Data-SIH-2.pdf, BIG-Data-SIH-3.pdf


In [ ]:
# Marine Biodiversity Chatbot with Ollama Llama 3.2 3B
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain_ollama import OllamaLLM
from langchain.prompts import PromptTemplate
import pickle
import json
from datetime import datetime

class MarineBiodiversityChatbot:
    def __init__(self, pdf_directory=COLAB_DATA_DIR, vectorstore_path=VECTORSTORE_DIR):
        self.pdf_directory = pdf_directory
        self.vectorstore_path = vectorstore_path
        self.documents = []
        self.vectorstore = None
        self.qa_chain = None
        self.llm = None

        # Marine domain knowledge
        self.marine_expertise = {
            "aquaculture": ["fish farming", "cultivation", "feeding", "growth", "health monitoring", "precision aquaculture"],
            "conservation": ["biodiversity", "ecosystem", "species protection", "habitat", "marine protected areas"],
            "ai_applications": ["computer vision", "machine learning", "predictive analytics", "automation", "sensor networks"],
            "ocean_monitoring": ["water quality", "environmental parameters", "data integration", "remote sensing", "GIS"],
            "sustainability": ["sustainable fishing", "ecosystem management", "climate change", "pollution control"]
        }

        print("🌊 Marine Biodiversity Chatbot initialized!")
        print(f"📁 PDF Directory: {self.pdf_directory}")
        print(f"💾 Vector Store: {self.vectorstore_path}")

    def setup_llm(self):
        """Initialize Ollama Llama 3.2 3B model"""
        print("🦙 Setting up Llama 3.2 3B model...")

        try:
            # Initialize Ollama LLM
            self.llm = OllamaLLM(
                model="llama3.2:3b",
                base_url="http://localhost:11434",
                temperature=0.3,
                top_p=0.9,
                repeat_penalty=1.1
            )

            # Test the model
            test_response = self.llm.invoke("Hello, I'm a marine science AI assistant.")
            print("✅ Llama 3.2 3B model ready!")
            print(f"🧪 Test response: {test_response[:100]}...")
            return True

        except Exception as e:
            print(f"❌ Error setting up Llama model: {e}")
            return False

    def load_marine_documents(self):
        """Load and process marine research PDFs"""
        print("\n📚 Loading marine research documents...")

        pdf_files = [f for f in os.listdir(self.pdf_directory) if f.endswith('.pdf')]

        if not pdf_files:
            print(f"❌ No PDF files found in {self.pdf_directory}")
            return False

        total_pages = 0
        for pdf_file in pdf_files:
            pdf_path = os.path.join(self.pdf_directory, pdf_file)
            print(f"📄 Processing: {pdf_file}")

            try:
                loader = PyPDFLoader(pdf_path)
                docs = loader.load()

                # Add source metadata
                for doc in docs:
                    doc.metadata['source_file'] = pdf_file
                    doc.metadata['domain'] = 'marine_science'

                self.documents.extend(docs)
                total_pages += len(docs)
                print(f"   ✅ Loaded {len(docs)} pages")

            except Exception as e:
                print(f"   ❌ Error loading {pdf_file}: {str(e)}")

        print(f"\n📊 Document Loading Summary:")
        print(f"   • Total PDFs processed: {len(pdf_files)}")
        print(f"   • Total pages loaded: {total_pages}")
        print(f"   • Documents in memory: {len(self.documents)}")

        return len(self.documents) > 0

    def create_vector_database(self):
        """Create FAISS vector database from marine documents"""
        print("\n🧠 Creating marine knowledge vector database...")

        if not self.documents:
            print("❌ No documents loaded. Run load_marine_documents() first.")
            return False

        # Advanced text splitting for better context preservation
        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=1500,  # Larger chunks for better context
            chunk_overlap=300,  # More overlap for continuity
            separators=["\n\n", "\n", ". ", "! ", "? ", " ", ""],
            keep_separator=True
        )

        print("✂️ Splitting documents into chunks...")
        text_chunks = text_splitter.split_documents(self.documents)

        print(f"📝 Created {len(text_chunks)} knowledge chunks")

        # Create embeddings with marine-optimized model
        print("🔤 Creating embeddings...")
        embeddings = HuggingFaceEmbeddings(
            model_name="sentence-transformers/all-MiniLM-L6-v2",
            model_kwargs={'device': 'cpu'}
        )

        # Create vector store
        print("🗄️ Building FAISS vector database...")
        self.vectorstore = FAISS.from_documents(text_chunks, embeddings)

        # Save vector store
        vectorstore_file = os.path.join(self.vectorstore_path, "marine_vectorstore")
        self.vectorstore.save_local(vectorstore_file)

        print(f"💾 Vector database saved to: {vectorstore_file}")
        print("✅ Marine knowledge database created successfully!")

        return True

    def setup_qa_chain(self):
        """Setup QA chain with marine-specific prompting"""
        print("\n🔗 Setting up marine expertise QA chain...")

        if not self.llm:
            print("❌ LLM not initialized. Run setup_llm() first.")
            return False

        if not self.vectorstore:
            print("❌ Vector store not created. Run create_vector_database() first.")
            return False

        # Marine-specific prompt template
        marine_prompt = PromptTemplate(
            input_variables=["context", "question"],
            template="""You are a Marine Biodiversity and Aquaculture AI Expert with deep knowledge of:
• AI applications in ocean conservation and marine research
• Advanced aquaculture technologies and precision farming
• Marine ecosystem monitoring and data integration
• Sustainable fisheries management and conservation strategies

Use the following research context to answer the question accurately and comprehensively:

RESEARCH CONTEXT:
{context}

QUESTION: {question}

EXPERT RESPONSE:
Provide a detailed, scientifically accurate answer based on the research context. Include:
1. Direct answers with specific details from the research
2. Practical applications and real-world examples
3. Technical explanations when relevant
4. Future implications and opportunities

If the question is outside the research scope, clearly state this and provide general marine science guidance.

Answer:"""
        )

        # Create QA chain
        self.qa_chain = RetrievalQA.from_chain_type(
            llm=self.llm,
            chain_type="stuff",
            retriever=self.vectorstore.as_retriever(
                search_type="similarity",
                search_kwargs={"k": 4}  # Retrieve top 4 most relevant chunks
            ),
            return_source_documents=True,
            chain_type_kwargs={"prompt": marine_prompt}
        )

        print("✅ Marine expertise QA chain ready!")
        return True

    def ask_marine_question(self, question: str):
        """Ask a question to the marine AI expert"""
        if not self.qa_chain:
            return {
                "answer": "❌ QA chain not initialized. Please run the full setup first.",
                "sources": [],
                "confidence": "low"
            }

        try:
            print(f"🔍 Analyzing question: {question}")

            # Get response from QA chain
            result = self.qa_chain({"query": question})

            # Extract source information
            sources = []
            for doc in result["source_documents"]:
                source_info = {
                    "file": doc.metadata.get("source_file", "Unknown"),
                    "page": doc.metadata.get("page", "Unknown"),
                    "content_preview": doc.page_content[:200] + "..."
                }
                sources.append(source_info)

            return {
                "answer": result["result"],
                "sources": sources,
                "confidence": "high",
                "timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            }

        except Exception as e:
            return {
                "answer": f"⚠️ Error processing question: {str(e)}",
                "sources": [],
                "confidence": "low"
            }

    def get_training_summary(self):
        """Get summary of training data and capabilities"""
        return {
            "documents_loaded": len(self.documents),
            "vector_database": "FAISS with sentence-transformers embeddings",
            "llm_model": "Ollama Llama 3.2 3B",
            "expertise_areas": list(self.marine_expertise.keys()),
            "status": "Ready for marine biodiversity and aquaculture questions"
        }

print("🌊 Marine Biodiversity Chatbot class defined!")
print("🚀 Ready for training and deployment!")


🌊 Marine Biodiversity Chatbot class defined!
🚀 Ready for training and deployment!


In [ ]:
# Complete training pipeline for Marine Biodiversity Chatbot
print("🌊 STARTING MARINE CHATBOT TRAINING PIPELINE")
print("=" * 60)

# Initialize chatbot
marine_bot = MarineBiodiversityChatbot()

# Step 1: Setup Llama 3.2 3B model
print("\n🦙 STEP 1: Setting up Llama 3.2 3B model...")
if marine_bot.setup_llm():
    print("✅ Llama 3.2 3B ready for marine expertise!")
else:
    print("❌ Failed to setup Llama model. Check Ollama installation.")
    raise Exception("LLM setup failed")

# Step 2: Load marine research documents
print("\n📚 STEP 2: Loading marine research documents...")
if marine_bot.load_marine_documents():
    print("✅ Marine research data loaded successfully!")
else:
    print("❌ Failed to load documents. Check PDF files.")
    raise Exception("Document loading failed")

# Step 3: Create vector database
print("\n🧠 STEP 3: Creating marine knowledge vector database...")
if marine_bot.create_vector_database():
    print("✅ Marine knowledge database created!")
else:
    print("❌ Failed to create vector database.")
    raise Exception("Vector database creation failed")

# Step 4: Setup QA chain
print("\n🔗 STEP 4: Setting up marine expertise QA chain...")
if marine_bot.setup_qa_chain():
    print("✅ Marine AI expert QA chain ready!")
else:
    print("❌ Failed to setup QA chain.")
    raise Exception("QA chain setup failed")

# Training completion
print("\n🎉 TRAINING COMPLETED SUCCESSFULLY!")
print("=" * 60)

# Display training summary
summary = marine_bot.get_training_summary()
print("\n📊 TRAINING SUMMARY:")
print(f"   • Documents processed: {summary['documents_loaded']}")
print(f"   • Vector database: {summary['vector_database']}")
print(f"   • LLM model: {summary['llm_model']}")
print(f"   • Expertise areas: {', '.join(summary['expertise_areas'])}")
print(f"   • Status: {summary['status']}")

print("\n🌊 Your Marine Biodiversity AI Expert is ready to answer questions!")


🌊 STARTING MARINE CHATBOT TRAINING PIPELINE
🌊 Marine Biodiversity Chatbot initialized!
📁 PDF Directory: /content/marine_data
💾 Vector Store: /content/vectorstore

🦙 STEP 1: Setting up Llama 3.2 3B model...
🦙 Setting up Llama 3.2 3B model...
✅ Llama 3.2 3B model ready!
🧪 Test response: That's fascinating! As a marine science AI assistant, you must have access to a vast amount of knowl...
✅ Llama 3.2 3B ready for marine expertise!

📚 STEP 2: Loading marine research documents...

📚 Loading marine research documents...
❌ No PDF files found in /content/marine_data
❌ Failed to load documents. Check PDF files.


Exception: Document loading failed

In [ ]:
# Test the trained Marine Biodiversity Chatbot
print("🧪 TESTING MARINE BIODIVERSITY AI EXPERT")
print("=" * 60)

# Comprehensive test questions covering different marine domains
test_questions = [
    # Aquaculture AI Applications
    "How can AI improve fish health monitoring in aquaculture systems?",

    # Marine Conservation Technology
    "What are the key AI technologies used for marine biodiversity conservation?",

    # Ocean Data Integration
    "How does GIS-based integration help in marine ecosystem management?",

    # Sustainable Fisheries
    "What role does AI play in preventing overfishing and promoting sustainable fisheries?",

    # Species Identification
    "How do computer vision techniques help in automated marine species identification?"
]

print(f"📋 Running {len(test_questions)} comprehensive tests...")
print("=" * 60)

# Run test questions
for i, question in enumerate(test_questions, 1):
    print(f"\n🔬 TEST {i}: {question}")
    print("-" * 80)

    # Get response from marine chatbot
    response = marine_bot.ask_marine_question(question)

    # Display response
    print("🤖 MARINE AI EXPERT RESPONSE:")
    print(response["answer"])

    # Show source information
    if response["sources"]:
        print(f"\n📚 Sources ({len(response['sources'])} references):")
        for j, source in enumerate(response["sources"], 1):
            print(f"   {j}. {source['file']} (Page: {source['page']})")

    print(f"\n✅ Confidence: {response['confidence']}")
    print(f"⏰ Generated at: {response['timestamp']}")
    print("=" * 80)

print("\n🎉 ALL TESTS COMPLETED!")
print("✅ Your Marine Biodiversity Chatbot is fully functional and ready for use!")


In [ ]:
# Interactive chat interface for Marine Biodiversity AI Expert
def start_marine_chat_interface():
    """Interactive chat with the marine biodiversity expert"""

    print("\n" + "🌊" * 30)
    print("WELCOME TO MARINE BIODIVERSITY AI EXPERT")
    print("🌊" * 30)
    print("\n🤖 Powered by Llama 3.2 3B + Marine Research Knowledge")
    print("📚 Trained on cutting-edge AI applications in marine science")

    print("\n🎯 I can help you with:")
    print("   🐟 AI applications in aquaculture and fish farming")
    print("   🌊 Marine conservation and ecosystem protection")
    print("   📊 Ocean data integration and GIS analysis")
    print("   🎣 Sustainable fisheries management")
    print("   🔬 Marine species identification technologies")
    print("   🤖 AI and machine learning in ocean research")

    print("\n💡 Example questions:")
    print("   • How can AI optimize fish feeding in aquaculture?")
    print("   • What computer vision methods are used for coral monitoring?")
    print("   • How does predictive analytics help marine conservation?")

    print("\n📝 Type 'quit' to exit the chat")
    print("🌊" + "-" * 58 + "🌊\n")

    chat_count = 0

    while True:
        try:
            # Get user input
            user_question = input("🌊 Your marine research question: ").strip()

            # Check for exit commands
            if user_question.lower() in ['quit', 'exit', 'bye', 'stop']:
                print("\n🐙 Thank you for exploring marine AI with us!")
                print("🌊 Keep protecting our oceans! 🌊")
                break

            # Skip empty questions
            if not user_question:
                print("❓ Please ask a question about marine science!")
                continue

            chat_count += 1
            print(f"\n🔍 [Chat {chat_count}] Consulting marine research database...")

            # Get response from chatbot
            response = marine_bot.ask_marine_question(user_question)

            # Display formatted response
            print("\n" + "🤖 MARINE AI EXPERT:")
            print("-" * 50)
            print(response["answer"])

            # Show sources if available
            if response["sources"]:
                print(f"\n📚 Research Sources:")
                for i, source in enumerate(response["sources"], 1):
                    print(f"   {i}. {source['file']}")

            print(f"\n🎯 Confidence: {response['confidence']}")
            print("=" * 80)

        except KeyboardInterrupt:
            print("\n\n🐙 Chat interrupted. Thanks for using Marine AI Expert!")
            break
        except Exception as e:
            print(f"\n⚠️ Error: {str(e)}")
            print("Please try asking your question again.")

# Start interactive chat
start_marine_chat_interface()
